# Rogers Revenue/Discount Analysis

In [66]:
import re
import pandas as pd
import datetime as dt
from datetime import date, timedelta
import matplotlib.pyplot as plt

## Import and Inspect Dataset Features

In [2]:
discount_data= pd.read_excel("sample_discount_data.xlsx", engine= 'openpyxl')
display(discount_data.dtypes)
nrows, ncols= discount_data.shape
print(f"Number of records: {nrows}\nNumber of columns: {ncols}")

COMPLIANT                      object
ACCOUNT_ID                      int64
OPEN_DATE              datetime64[ns]
SERVICE_PROVINCE               object
DISCOUNT_CODE                  object
DISCOUNT_START_DATE            object
DISCOUNT_END_DATE              object
AMOUNT                        float64
ORDER                           int64
ORDER_DATE                     object
ORDER_TYPE                     object
SOURCE_SYSTEM                  object
dtype: object

Number of records: 3517
Number of columns: 12


### Clean Dataset

In [3]:
# Function to clean dataset
date_format1= lambda x: dt.datetime.strptime(x, "%b %d %Y").strftime("%Y-%m-%d")
date_format2= lambda x: dt.datetime.strptime(x, "%B %d %Y").strftime("%Y-%m-%d")

date_pat1= r"\b[A-Za-z]{3}\s{1,}\d{1,2}\s\d{4}\b"
date_pat2= r"\b[A-Za-z]{3,}\s{1,}\d{1,2}\s\d{4}\b"

province_map= {"ON": "ONTARIO",
              "ONT":"ONTARIO",
              "ONT.":"ONTARIO",
              "ON.":"ONTARIO",
              "ONTARIO": "ONTARIO",
              "NB":"NEW BRUNSWICK",
              "nb.":"NEW BRUNSWICK",
              "New_Brunswick":"NEW BRUNSWICK",
              "NEW BRUNSWICK":"NEW BRUNSWICK",
              "NL":"NEWFOUNDLAND AND LABRADOR",
              "Newfoundland & Labrador":"NEWFOUNDLAND AND LABRADOR",
              "Newfoundland and Labrador":"NEWFOUNDLAND AND LABRADOR"
              }

def clean_data(df):
    df1= df.copy()

    # 1. Treat date and datetime object: convert to homogenous format
    # convert open_date to date object
    df1["OPEN_DATE"]= df1.OPEN_DATE.apply(lambda x: x.strftime("%Y-%m-%d"))
    
    # convert discount_start_date
    # convert cases with "1st", "2nd", "3rd", "4th"
    df1.DISCOUNT_START_DATE= df1.DISCOUNT_START_DATE.str.replace(r"st|nd|th|rd", "", regex= True)
    # drop extra spaces in the middle of dates
    df1.DISCOUNT_START_DATE= df1.DISCOUNT_START_DATE.str.replace(r"\s{2,}", " ", regex= True)
    # drop commas
    df1.DISCOUNT_START_DATE= df1.DISCOUNT_START_DATE.str.replace(",", "", regex= True)
    # fix typo "Augu"
    df1.DISCOUNT_START_DATE= df1.DISCOUNT_START_DATE.str.replace("Augu ", "Aug ")
    
    # change format to YYYY-MM-DD
    df1.loc[(df1.DISCOUNT_START_DATE.str.contains(date_pat1, regex= True)), "DISCOUNT_START_DATE"]=\
    df1[(df1.DISCOUNT_START_DATE.str.contains(date_pat1, regex=True))].DISCOUNT_START_DATE.apply(date_format1)
    
    df1.loc[(df1.DISCOUNT_START_DATE.str.contains(date_pat2, regex= True)), "DISCOUNT_START_DATE"]=\
    df1[(df1.DISCOUNT_START_DATE.str.contains(date_pat2, regex=True))].DISCOUNT_START_DATE.apply(date_format2)
    
    # format discount_end_date
    df1.DISCOUNT_END_DATE= df1.DISCOUNT_END_DATE.replace(r"[\W|\D]","-", regex=True)
    
    # convert and format order_date
    # strip leading and laging "<,>"
    df1.ORDER_DATE= df1.ORDER_DATE.replace(r"[\W|\D]","", regex=True)
    df1.ORDER_DATE= df1.ORDER_DATE.astype('int64')
    
    # define excel start date
    start_dt= dt.datetime(1899, 12, 30)
    # convert to YYYY-MM-DD format
    df1["ORDER_DATE"]= df1.ORDER_DATE.apply(lambda n:(start_dt + timedelta(n)).strftime("%Y-%m-%d"))
    
    # 2. Map provinces: Create homogenous format
    df1["SERVICE_PROVINCE"]= df1["SERVICE_PROVINCE"].str.strip().map(province_map)
    
    # format unique IDs to keep format: i.e. 0000xxx
    df1['ACCOUNT_ID']= df1['ACCOUNT_ID'].astype('str').apply(lambda x: x.zfill(9))
    
    # change discount values to positive
    df1['DISCOUNT_PERC']= df1['AMOUNT']*-1
    
    return df1

In [4]:
discount_clean= clean_data(discount_data)

discount_clean.shape

(3517, 13)

### Q1. How Many Complaint Orders?

In [21]:
# drop duplicate orders/records across dataset
#discount_clean.drop_duplicates(subset=['COMPLIANT', 'ACCOUNT_ID','DISCOUNT_CODE','AMOUNT','ORDER','ORDER_DATE','ORDER_TYPE','SOURCE_SYSTEM'], inplace=True)

discount_clean.drop_duplicates(subset= ['ORDER_DATE','ACCOUNT_ID','ORDER', 'COMPLIANT'], inplace=True)
discount_clean.shape

(3012, 13)

In [22]:
print("For the Month of December 2023")
print("- - - - - - - - - - - - - - - - - - - - - - \n")
print(f"Number of Compliant Orders: {discount_clean[discount_clean.COMPLIANT=='Y'].ORDER.nunique()}")
print(f"Number of Non-Compliant Orders: {discount_clean[discount_clean.COMPLIANT=='N'].ORDER.nunique()}\n")

print(f"Number of Accounts With Compliant Orders: {discount_clean[discount_clean.COMPLIANT=='Y'].ACCOUNT_ID.nunique()}")
print(f"Number of Accounts With Compliant Orders: {discount_clean[discount_clean.COMPLIANT=='N'].ACCOUNT_ID.nunique()}")

For the Month of December 2023
- - - - - - - - - - - - - - - - - - - - - - 

Number of Compliant Orders: 1865
Number of Non-Compliant Orders: 1147

Number of Accounts With Compliant Orders: 1523
Number of Accounts With Compliant Orders: 994


### Q2. How much compliant/ non-compliant unique orders across each province?

In [24]:
# tally orders by province
order_by_prov= discount_clean[['SERVICE_PROVINCE','ACCOUNT_ID','ORDER']].groupby('SERVICE_PROVINCE').\
                        aggregate(Total_Accounts= ('ACCOUNT_ID', lambda x: x.nunique()),
                                 Total_Orders= ('ORDER', lambda x: x.nunique())).reset_index()

order_by_prov

,SERVICE_PROVINCE,Total_Accounts,Total_Orders
0,NEW BRUNSWICK,241,250
1,NEWFOUNDLAND AND LABRADOR,44,44
2,ONTARIO,2055,2718


In [73]:
# build pivot to investigate features against province and compliance
province_pivot= pd.pivot_table(discount_clean, values=['DISCOUNT_PERC', 'ORDER', 'ACCOUNT_ID'], index=['SERVICE_PROVINCE', 'COMPLIANT'],
                       aggfunc={'DISCOUNT_PERC': ["min", "mean", "max"],
                                'ORDER': lambda x: x.nunique(),
                                'ACCOUNT_ID': lambda x: x.nunique()}).reset_index().\
                merge(order_by_prov,'left', on='SERVICE_PROVINCE')

column_map= {('COMPLIANT', ''): "COMPLIANT", ('ACCOUNT_ID', '<lambda>'): "ACCOUNTS",\
            ('DISCOUNT_PERC', 'max'): 'DISCOUNT_PERC_MAX',
            ('DISCOUNT_PERC', 'min'): 'DISCOUNT_PERC_MIN',
            ('DISCOUNT_PERC', 'mean'): 'DISCOUNT_PERC_AVG',
            ('ORDER', '<lambda>'): "ORDERS"}

province_pivot['COMP_ACC_PERC']= province_pivot[('ACCOUNT_ID', '<lambda>')]/province_pivot['Total_Accounts']
province_pivot['COMP_ORDER_PERC']= province_pivot[('ORDER', '<lambda>')]/province_pivot['Total_Orders']

province_pivot= province_pivot[['SERVICE_PROVINCE',('COMPLIANT', ''), ('ACCOUNT_ID', '<lambda>'),'COMP_ACC_PERC',('DISCOUNT_PERC', 'max'),\
               ('DISCOUNT_PERC', 'mean'),('DISCOUNT_PERC', 'min'),('ORDER', '<lambda>'),'COMP_ORDER_PERC']]

province_pivot.rename(columns= column_map, inplace= True)

/var/folders/0b/j5_tfk6s0lx6w21rfw19l9cw0000gp/T/ipykernel_37113/1951542675.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  province_pivot= pd.pivot_table(discount_clean, values=['DISCOUNT_PERC', 'ORDER', 'ACCOUNT_ID'], index=['SERVICE_PROVINCE', 'COMPLIANT'],
/Users/Olumedey/Documents/Programming Projects/Python Projects/venv/lib/python3.8/site-packages/pandas/core/common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


### Q3. Compliance Across Discount Codes

In [26]:
# tally order count by province and discount code
order_by_disc_prov= discount_clean[['SERVICE_PROVINCE','ORDER','ACCOUNT_ID','DISCOUNT_CODE']].groupby(['SERVICE_PROVINCE','DISCOUNT_CODE']).\
                        aggregate(Total_Accounts= ('ACCOUNT_ID', lambda x: x.nunique()),
                                 Total_Orders= ('ORDER', lambda x: x.nunique())).reset_index()

order_by_disc_prov

,SERVICE_PROVINCE,DISCOUNT_CODE,Total_Accounts,Total_Orders
0,NEW BRUNSWICK,EKK,126,127
1,NEW BRUNSWICK,ZAK,121,123
2,NEWFOUNDLAND AND LABRADOR,EKK,26,26
3,NEWFOUNDLAND AND LABRADOR,ZAK,18,18
4,ONTARIO,FQK,1793,2313
5,ONTARIO,LDK,390,405


In [55]:
# build pivot to inspect features against discount codes
code_pivot= pd.pivot_table(discount_clean, values=['DISCOUNT_PERC', 'ORDER', 'ACCOUNT_ID'], index=['DISCOUNT_CODE','SERVICE_PROVINCE','COMPLIANT'],
                       aggfunc={'DISCOUNT_PERC': ["min", "mean","max"],
                                'ORDER': lambda x: x.nunique(),
                                'ACCOUNT_ID': lambda x: x.nunique()}).reset_index().\
                merge(order_by_disc_prov,'left', on=['SERVICE_PROVINCE','DISCOUNT_CODE'])

column_map= {('COMPLIANT', ''): "COMPLIANT", ('ACCOUNT_ID', '<lambda>'): "ACCOUNTS",\
            ('DISCOUNT_PERC', 'max'): 'MAX_DISCOUNT_PERC',
            ('DISCOUNT_PERC', 'min'): 'MIN_DISCOUNT_PERC',
            ('DISCOUNT_PERC', 'mean'): 'AVG_DISCOUNT_PERC',
            ('ORDER', '<lambda>'): "ORDER_COUNT"}

code_pivot['COMP_ACC_PERC']= code_pivot[('ACCOUNT_ID', '<lambda>')]/code_pivot['Total_Accounts']
code_pivot['COMP_ORDER_PERC']= code_pivot[('ORDER', '<lambda>')]/code_pivot['Total_Orders']

code_pivot= code_pivot[['DISCOUNT_CODE', 'SERVICE_PROVINCE',('COMPLIANT', ''), ('ACCOUNT_ID', '<lambda>'),'COMP_ACC_PERC',('DISCOUNT_PERC', 'max'),\
               ('DISCOUNT_PERC', 'mean'),('DISCOUNT_PERC', 'min'),('ORDER', '<lambda>'),'COMP_ORDER_PERC']]

code_pivot.rename(columns= column_map, inplace= True)#.sort_values(['SERVICE_PROVINCE','DISCOUNT_CODE'])
code_pivot

/var/folders/0b/j5_tfk6s0lx6w21rfw19l9cw0000gp/T/ipykernel_37113/3520999748.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  code_pivot= pd.pivot_table(discount_clean, values=['DISCOUNT_PERC', 'ORDER', 'ACCOUNT_ID'], index=['DISCOUNT_CODE','SERVICE_PROVINCE','COMPLIANT'],
/Users/Olumedey/Documents/Programming Projects/Python Projects/venv/lib/python3.8/site-packages/pandas/core/common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


,DISCOUNT_CODE,SERVICE_PROVINCE,COMPLIANT,ACCOUNTS,COMP_ACC_PERC,MAX_DISCOUNT_PERC,AVG_DISCOUNT_PERC,MIN_DISCOUNT_PERC,ORDER_COUNT,COMP_ORDER_PERC
0,EKK,NEW BRUNSWICK,N,40,0.317460,5.0,5.000000,5.00,40,0.314961
1,EKK,NEW BRUNSWICK,Y,86,0.682540,5.0,5.000000,5.00,87,0.685039
2,EKK,NEWFOUNDLAND AND LABRADOR,N,19,0.730769,5.0,5.000000,5.00,19,0.730769
3,EKK,NEWFOUNDLAND AND LABRADOR,Y,7,0.269231,5.0,5.000000,5.00,7,0.269231
4,FQK,ONTARIO,N,780,0.435025,10.0,9.981189,7.68,883,0.381755
5,FQK,ONTARIO,Y,1207,0.673173,10.0,9.985944,7.76,1430,0.618245
6,LDK,ONTARIO,N,165,0.423077,10.0,10.000000,10.00,168,0.414815
7,LDK,ONTARIO,Y,232,0.594872,10.0,10.000000,10.00,237,0.585185
8,ZAK,NEW BRUNSWICK,N,24,0.198347,5.0,5.000000,5.00,24,0.195122
9,ZAK,NEW BRUNSWICK,Y,97,0.801653,5.0,5.000000,5.00,99,0.804878


### Q4. Compliance Across Order Types and Source Systems

In [41]:
# Investigate orders by Order_Type and Source_Systems

display(discount_clean[['SOURCE_SYSTEM','ORDER','ACCOUNT_ID']].groupby(['SOURCE_SYSTEM']).\
                        aggregate(TOTAL_ACCOUNTS= ('ACCOUNT_ID', lambda x: x.nunique()),
                                 TOTAL_ORDERS= ('ORDER', lambda x: x.nunique())).reset_index(),\
        discount_clean[['ORDER_TYPE','ORDER','ACCOUNT_ID']].groupby(['ORDER_TYPE']).\
                        aggregate(TOTAL_ACCOUNTS= ('ACCOUNT_ID', lambda x: x.nunique()),
                                 TOTAL_ORDERS= ('ORDER', lambda x: x.nunique())).reset_index())

,SOURCE_SYSTEM,TOTAL_ACCOUNTS,TOTAL_ORDERS
0,CRM,473,499
1,Digital,1922,2513


,ORDER_TYPE,TOTAL_ACCOUNTS,TOTAL_ORDERS
0,CH,109,110
1,CM,2,2
2,ES,8,8
3,PR,2133,2891
4,SW,1,1


In [35]:
# tally order count by type and source
otype_source= discount_clean[['ORDER_TYPE','SOURCE_SYSTEM','ORDER','ACCOUNT_ID']].groupby(['ORDER_TYPE','SOURCE_SYSTEM']).\
                        aggregate(TOTAL_ACCOUNTS= ('ACCOUNT_ID', lambda x: x.nunique()),
                                 TOTAL_ORDERS= ('ORDER', lambda x: x.nunique())).reset_index()

otype_source

,ORDER_TYPE,SOURCE_SYSTEM,TOTAL_ACCOUNTS,TOTAL_ORDERS
0,CH,CRM,106,107
1,CH,Digital,3,3
2,CM,CRM,2,2
3,ES,CRM,8,8
4,PR,CRM,361,381
5,PR,Digital,1920,2510
6,SW,CRM,1,1


In [30]:
# build pivot to inspect features against source_system and order_type
otype_pivot= pd.pivot_table(discount_clean, values=['ORDER', 'ACCOUNT_ID'], index=['SOURCE_SYSTEM','ORDER_TYPE','COMPLIANT'],
                       aggfunc={'ORDERS_CNT': lambda x: x.nunique(),
                                'ACCOUNT_ID': lambda x: x.nunique()}).reset_index().\
                        merge(otype_source,'left', on=['SOURCE_SYSTEM','ORDER_TYPE'])

otype_pivot['COMP_ACC_PERC']= otype_pivot['ACCOUNT_ID']/otype_pivot['TOTAL_ACCOUNTS']
otype_pivot['COMP_ORDER_PERC']= otype_pivot['ORDERS_CNT']/otype_pivot['TOTAL_ORDERS']


otype_pivot= otype_pivot[['SOURCE_SYSTEM', 'ORDER_TYPE', 'COMPLIANT','ACCOUNT_ID','COMP_ACC_PERC',\
                          'ORDERS_CNT','COMP_ORDER_PERC']]

otype_pivot

,SOURCE_SYSTEM,ORDER_TYPE,COMPLIANT,ACCOUNT_ID,COMP_ACC_PERC,ORDER,COMP_ORDER_PERC
0,CRM,CH,N,43,0.405660,43,0.401869
1,CRM,CH,Y,63,0.594340,64,0.598131
2,CRM,CM,N,2,1.000000,2,1.000000
3,CRM,ES,N,1,0.125000,1,0.125000
4,CRM,ES,Y,7,0.875000,7,0.875000
5,CRM,PR,N,161,0.445983,164,0.430446
6,CRM,PR,Y,213,0.590028,217,0.569554
7,CRM,SW,Y,1,1.000000,1,1.000000
8,Digital,CH,N,1,0.333333,1,0.333333
9,Digital,CH,Y,2,0.666667,2,0.666667


### Q5. Revenue Leakage Points or Discrepancies? 

In [65]:
# build pivot to inspect features against unique discount codes, discounts
disc_timing_pivot= pd.pivot_table(discount_clean, values=['ORDER_DATE','OPEN_DATE','DISCOUNT_START_DATE','DISCOUNT_END_DATE'], index=['SERVICE_PROVINCE','DISCOUNT_CODE','DISCOUNT_PERC', 'COMPLIANT'],
                       aggfunc={'DISCOUNT_START_DATE': ["min","max"],
                                'DISCOUNT_END_DATE': ["min","max"],
                                'ORDER_DATE': ["min","max"],
                               'OPEN_DATE':["min","max"]}).reset_index()

column_map= {(   'SERVICE_PROVINCE',    ''): 'SERVICE_PROVINCE',
            (      'DISCOUNT_CODE',    ''): 'DISCOUNT_CODE',
            (      'DISCOUNT_PERC',    ''): 'DISCOUNT_PERC',
            (          'COMPLIANT',    ''): 'COMPLIANT',
            (  'DISCOUNT_END_DATE', 'max'): 'MAX_DISCOUNT_END_DATE',
            (  'DISCOUNT_END_DATE', 'min'): 'MIN_DISCOUNT_END_DATE',
            ('DISCOUNT_START_DATE', 'max'): 'MAX_DISCOUNT_START_DATE',
            ('DISCOUNT_START_DATE', 'min'): 'MIN_DISCOUNT_START_DATE',
            (          'OPEN_DATE', 'max'): 'MAX_OPEN_DATE',
            (          'OPEN_DATE', 'min'): 'MIN_OPEN_DATE',
            (         'ORDER_DATE', 'max'): 'MAX_ORDER_DATE',
            (         'ORDER_DATE', 'min'): 'MIN_ORDER_DATE'}

disc_timing_pivot.columns= disc_timing_pivot.columns.map(column_map)

# Preview non-compliant Discount codes and their dates
disc_timing_pivot[disc_timing_pivot.COMPLIANT=='N']

,SERVICE_PROVINCE,DISCOUNT_CODE,DISCOUNT_PERC,COMPLIANT,MAX_DISCOUNT_END_DATE,MIN_DISCOUNT_END_DATE,MAX_DISCOUNT_START_DATE,MIN_DISCOUNT_START_DATE,MAX_OPEN_DATE,MIN_OPEN_DATE,MAX_ORDER_DATE,MIN_ORDER_DATE
0,NEW BRUNSWICK,EKK,5.00,N,2027-01-23,2026-12-06,2023-12-31,2023-12-06,2023-12-31,2020-08-07,2023-12-31,2023-12-03
2,NEW BRUNSWICK,ZAK,5.00,N,2025-01-17,2024-12-07,2023-12-31,2023-12-06,2023-12-29,2019-12-06,2023-12-29,2023-12-01
4,NEWFOUNDLAND AND LABRADOR,EKK,5.00,N,2027-01-22,2026-12-06,2024-01-01,2023-12-06,2023-12-21,2020-11-09,2023-12-24,2023-12-01
6,NEWFOUNDLAND AND LABRADOR,ZAK,5.00,N,2025-01-14,2024-12-06,2023-12-29,2023-12-06,2023-12-28,2021-01-22,2023-12-28,2023-12-01
8,ONTARIO,FQK,7.68,N,2025-12-01,2025-12-01,2023-12-02,2023-12-02,2020-12-03,2020-12-03,2023-12-02,2023-12-02
10,ONTARIO,FQK,7.84,N,2025-12-31,2025-12-31,2023-12-31,2023-12-31,2023-12-29,2023-12-29,2023-12-29,2023-12-29
11,ONTARIO,FQK,7.92,N,2026-01-15,2026-01-15,2023-12-22,2023-12-22,2019-11-22,2019-11-22,2023-12-22,2023-12-22
13,ONTARIO,FQK,8.16,N,2026-01-10,2026-01-10,2023-12-12,2023-12-12,2023-07-19,2023-07-19,2023-12-12,2023-12-12
14,ONTARIO,FQK,8.24,N,2025-12-20,2025-12-20,2023-12-07,2023-12-07,2020-10-29,2020-10-29,2023-12-07,2023-12-07
15,ONTARIO,FQK,8.33,N,2026-01-10,2026-01-10,2023-12-13,2023-12-13,2023-01-03,2023-01-03,2023-12-13,2023-12-13


In [69]:
# tally order counts by day
daily_orders= discount_clean[['ORDER_DATE','ORDER']].groupby('ORDER_DATE').\
                aggregate(TOTAL_ORDERS= ('ORDER', lambda x: x.nunique())).reset_index()

daily_orders
# display(daily_orders, plt.plot(daily_orders.ORDER_DATE,daily_orders.TOTAL_ORDERS))

,ORDER_DATE,TOTAL_ORDERS
0,2023-12-01,68
1,2023-12-02,72
2,2023-12-03,44
3,2023-12-04,88
4,2023-12-05,75
5,2023-12-06,65
6,2023-12-07,77
7,2023-12-08,64
8,2023-12-09,65
9,2023-12-10,40


In [70]:
# build pivot to inspect features by order dates
dates_pivot= pd.pivot_table(discount_clean, values=['DISCOUNT_PERC', 'ORDER', 'DISCOUNT_END_DATE'], index=['ORDER_DATE','DISCOUNT_CODE','SERVICE_PROVINCE','COMPLIANT'],
                       aggfunc={'DISCOUNT_PERC': "mean",
                                'ORDER': lambda x: x.nunique()}).reset_index()\
                        .merge(daily_orders,'left', on=['ORDER_DATE'])


dates_pivot['DAILY_ORDER_PERC']= dates_pivot['ORDER']/dates_pivot['TOTAL_ORDERS']

dates_pivot= dates_pivot[['ORDER_DATE', 'DISCOUNT_CODE', 'SERVICE_PROVINCE', 'COMPLIANT',
               'DISCOUNT_PERC', 'ORDER', 'TOTAL_ORDERS','DAILY_ORDER_PERC']]

dates_pivot

,ORDER_DATE,DISCOUNT_CODE,SERVICE_PROVINCE,COMPLIANT,DISCOUNT_PERC,ORDER,TOTAL_ORDERS,DAILY_ORDER_PERC
0,2023-12-01,EKK,NEW BRUNSWICK,Y,5.0,3,68,0.044118
1,2023-12-01,EKK,NEWFOUNDLAND AND LABRADOR,N,5.0,1,68,0.014706
2,2023-12-01,FQK,ONTARIO,N,10.0,9,68,0.132353
3,2023-12-01,FQK,ONTARIO,Y,10.0,35,68,0.514706
4,2023-12-01,LDK,ONTARIO,N,10.0,6,68,0.088235
...,...,...,...,...,...,...,...,...
242,2023-12-31,EKK,NEW BRUNSWICK,N,5.0,1,37,0.027027
243,2023-12-31,FQK,ONTARIO,N,10.0,15,37,0.405405
244,2023-12-31,FQK,ONTARIO,Y,10.0,18,37,0.486486
245,2023-12-31,LDK,ONTARIO,N,10.0,2,37,0.054054


## Export Datasets for Dashboard Reporting

In [74]:
# Load pivots into Excel workbook/report

with pd.ExcelWriter("discount_data_report.xlsx") as writer:  
    discount_clean.to_excel(writer, sheet_name='Discount_Data_Clean')
    dates_pivot.to_excel(writer, sheet_name='Daily_Orders')
    province_pivot.to_excel(writer, sheet_name='Province_Pivot')
    code_pivot.to_excel(writer, sheet_name='Discount_Code_Pivot')
    otype_pivot.to_excel(writer, sheet_name='Order_Type_Pivot')
    disc_timing_pivot.to_excel(writer, sheet_name='Date_Pivot')